# 一千零一夜主题分析 Topic modeling analysis on 'arabian_nights' dataset

为了学习LDA主题分析模型，这里选了一千零一夜的1000 个（别问我为什么不是1001个）txt文本做主题分析，目的是从中提取出10个主题，在不读故事的情况下简单了解一下这1000个故事大概涵盖了哪些内容。

1.定义函数，获取文件夹内所有文件

In [8]:
def list_textfiles(directory):
    "Return a list of filenames ending in '.txt' in DIRECTORY."
    textfiles = []
    for filename in listdir(directory):
        if filename.endswith(".txt"):
            textfiles.append(directory + "/" + filename)
    return textfiles

2.定义函数，读取文件内容（故事）

In [43]:

def read_file(filename):
    "Read the contents of FILENAME and return as a string."
    infile = open(filename) # windows users should use codecs.open after importing codecs
    contents = infile.read()
    infile.close()
    return contents

3.分词并去掉停用词

In [40]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

files = list_textfiles('arabian_nights')
textfiles_tokenized = []
for f in files:
    tokens = word_tokenize(read_file(f).decode('utf-8'))
    filtered_words = [word for word in tokens if word not in stopwords.words('english')]
    textfiles_tokenized.append(' '.join(filtered_words))


4.只从文本中提取1000个最重要的特征关键词，然后停止。
开始关键词提取和向量转换过程：

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(textfiles_tokenized)#参数应当是iterable的list

5.引入LDA，并设定（懵一下）主题个数为10，执行提取主题

In [44]:
from sklearn.decomposition import LatentDirichletAllocation
n_components = 10
lda = LatentDirichletAllocation(n_components=n_components, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

#执行提取主题
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

6.定义一个函数，输出每个主题的前十个关键词

In [45]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


In [38]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
sharrkan zau makan brother dandan kanmakan wazir omar chamberlain bin uman nu moslems ye baghdad nuzhat army infidels sultan horse
Topic #1:
gharib city horse brother thousand sa ye janshah prince bade days palace land father mounted ad sword fared earth god
Topic #2:
lay zaman silk kamar house daughter sat sleep ready old seventy began drew years cloth repeated filled koran moon feet
Topic #3:
din nur sea ship city ala merchants island gold captain water bulukiya behold thousand shaykh miriam goods hasib days ate
Topic #4:
fish fisherman judar jew net khalifah ape bags saddle mother cast mule brothers bring bread eat brother dish wilt good
Topic #5:
couplets lady tears face er fair wept beauty sight lover muluk wine moon verses weeping youth sleep garden life recited
Topic #6:
wazir father hasan wife daughter mother zaman kamar palace queen city princess kissed bade minister sultan sister badr marriage old
Topic #7:
caliph house abu ali faithful dinars commander slave door t

7.做交互图展示结果

In [46]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/Users/zjm/anaconda/lib/python2.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      27.225928        1       1  0.038103 -0.037945
7      17.946158        1       2  0.037413  0.000977
5      16.788624        1       3 -0.064086 -0.035127
8      12.847429        1       4  0.023046 -0.036667
3      12.561114        1       5  0.037744  0.003831
1       4.784838        1       6  0.269949  0.137431
6       2.893650        1       7 -0.013346 -0.242018
2       2.667263        1       8 -0.144541  0.113255
4       2.242462        1       9 -0.219630  0.107866
0       0.042534        1      10  0.035349 -0.011606, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
392   Default   683.000000      hasan   683.000000  30.0000  30.0000
225   Default   774.000000        din   774.000000  29.0000  29.0000
7     Default   660.000000        abu   660.000000  28.0000  28.0000
33    Default   573.000000        ali   573.000000  27.0000  27.0000
129   Default  1065.000000     caliph  1065.000000  26.0000  26.0000
570   Default   792.000000     mother   792.000000  25.0000  25.0000
299   Default  1223.000000     father  1223.000000  24.0000  24.0000
184   Default   670.000000   couplets   670.000000  23.0000  23.0000
996   Default   395.000000      zaman   395.000000  22.0000  22.0000
863   Default   541.000000      tears   541.000000  21.0000  21.0000
345   Default   579.000000     gharib   579.000000  20.0000  20.0000
937   Default  1205.000000      wazir  1205.000000  19.0000  19.0000
234   Default   848.000000       door   848.000000  18.0000  18.0000
543   Default   583.000000   merchant   583.000000  17.0000  17.0000
446   Default   299.000000      kamar   299.000000  16.0000  16.0000
956   Default   906.000000       wife   906.000000  15.0000  15.0000
288   Default   549.000000   faithful   549.000000  14.0000  14.0000
944   Default   584.000000       wept   584.000000  13.0000  13.0000
227   Default   589.000000     dinars   589.000000  12.0000  12.0000
419   Default  1301.000000      house  1301.000000  11.0000  11.0000
173   Default   500.000000  commander   500.000000  10.0000  10.0000
95    Default   470.000000        bin   470.000000   9.0000   9.0000
30    Default   240.000000        ala   240.000000   8.0000   8.0000
268   Default   389.000000         er   389.000000   7.0000   7.0000
471   Default   954.000000       lady   954.000000   6.0000   6.0000
444   Default   218.000000      judar   218.000000   5.0000   5.0000
735   Default   579.000000        sea   579.000000   4.0000   4.0000
121   Default  1039.000000    brother  1039.000000   3.0000   3.0000
878   Default   904.000000   thousand   904.000000   2.0000   2.0000
162   Default  1181.000000       city  1181.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
763   Topic10     0.099566      shams   114.770320   0.7128  -6.9055
295   Topic10     0.099349     faring    93.837464   0.9119  -6.9077
428   Topic10     0.099588   infidels   120.780431   0.6619  -6.9053
816   Topic10     0.099391      spake    99.708905   0.8517  -6.9073
345   Topic10     0.100589     gharib   579.595011  -0.8964  -6.8953
765   Topic10     0.100449   sharrkan   348.366979  -0.3887  -6.8967
997   Topic10     0.099952        zau   279.078472  -0.1719  -6.9017
521   Topic10     0.099914      makan   278.033572  -0.1686  -6.9020
121   Topic10     0.099838    brother  1039.074954  -1.4877  -6.9028
988   Topic10     0.099806         ye   878.809024  -1.3205  -6.9031
569   Topic10     0.099795    moslems   217.538872   0.0756  -6.9032
419   Topic10     0.099726      house  1301.152420  -1.7137  -6.9039
937   Topic10     0.099667      wazir  1205.650165  -1.6381  -6.9045
29    Topic10     0.099654       ajib   145.609948   0.4756  -6.9047
95    Topic10     0.099645        bin   470.058235  -0.6964  -6.90

notes：
1. 图的左侧，用圆圈代表不同的主题，圆圈的大小代表了每个主题分别包含文章的数量。
2. 图的右侧，列出了最重要（频率最高）的30个关键词列表。注意当你没有把鼠标悬停在任何主题之上的时候，这30个关键词代表全部文本中提取到的30个最重要关键词。
3. 每个主题表示什么意义？ 通过鼠标悬浮在左边的气泡上，我们可以选择查看具体的某个主题。选定后，右侧面板会相应地显示出跟这个主题相关的词汇，通过总结这些词汇表达的意义，我们可以归纳出该主题的意义
4. 但是，作者却不局限于此，右侧词汇都是跟主题相关，但是到底谁更相关呢？在这里，LDAvis的作者提出了一个算法：
relevance(term w | topic t) = λ * p(w | t) + (1 - λ) * p(w | t)/p(w);
某个词语主题的相关性，由λ参数来调节。如果λ接近1，那么在该主题下更频繁出现的词，跟主题更相关；如果λ越接近0，那么该主题下更特殊、更独有（exclusive）的词，跟主题更相关。

例如topic 1：
λ=1时，top 6 words是 wazir（(伊斯兰教国家元老,高官)维齐尔）,woman, old, wife,father, daughter; 猜测可能是家庭相关主题

λ=0时，top 6 words是 nurse,shimas（一个wazir的名字）,muluk（名字）,justice,counsel（忠告，劝告）,affairs（绯闻韵事）;

所以，结合两种情况，topic1 很可能是有关家庭伦理类的故事内容。

再看一个和topic 7，和其他主题相距较远，独居图的下方，表示与其他主题相关性较小：
λ=1时，top 6 words是 zaman,kamar,ma,ring,aruf,eunuch（宦官）; ???

λ=0时，top 6 words是 ma,aruf,budur,kamr,zaman,ring. ???

搜索上下文得知Ma'aruf,zaman,kamar都是人名，猜测这个主题主要是讲他们几个人的故事，再结合排名较后的关键词比如jeweller，merchant等，猜测这一主题还可能与珠宝商相关。



参考文献：http://www.jianshu.com/p/fdde9fc03f94
特别感谢这篇参考文献，教一个小白初步了解LDA模型，尤其是交互图的实现，特别直观特别炫。